<a href="https://colab.research.google.com/github/H3adlock/H3adlock/blob/master/SemanticsPythonDataAnalystTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scrapy

In [0]:
!ls -l
!scrapy startproject sprtprsn
!ls -l

total 8
drwxr-xr-x 1 root root 4096 Jan  8 17:15 sample_data
drwxr-sr-x 3 root root 4096 Jan 12 08:28 sprtprsn
Error: scrapy.cfg already exists in /content/sprtprsn
total 8
drwxr-xr-x 1 root root 4096 Jan  8 17:15 sample_data
drwxr-sr-x 3 root root 4096 Jan 12 08:28 sprtprsn


In [0]:
!ls -l sprtprsn/sprtprsn

total 60
-rw-r--r-- 1 root root 14498 Jan 12 11:31 female_sprtprsn.csv
-rw-r--r-- 1 root root     0 Jan 12 08:28 __init__.py
-rw-r--r-- 1 root root   205 Jan 12 10:49 items.py
-rw-r--r-- 1 root root 19445 Jan 12 11:31 male_sprtprsn.csv
-rw-r--r-- 1 root root  3601 Jan 12 08:28 middlewares.py
-rw-r--r-- 1 root root   289 Jan 12 08:28 pipelines.py
drwxr-sr-x 2 root root  4096 Jan 12 10:50 __pycache__
-rw-r--r-- 1 root root  3094 Jan 12 08:28 settings.py
drwxr-sr-x 3 root root  4096 Jan 12 08:29 spiders


In [0]:
import os
os.chdir('sprtprsn/sprtprsn')

In [0]:
%%writefile items.py
import scrapy
class sprtprsnItem(scrapy.Item):
    iD = scrapy.Field()
    name = scrapy.Field()
    gender = scrapy.Field()
    height = scrapy.Field()
    weight = scrapy.Field()
    url = scrapy.Field()


Overwriting items.py


In [0]:
!cat items.py

import scrapy
class sprtprsnItem(scrapy.Item):
    iD = scrapy.Field()
    name = scrapy.Field()
    gender = scrapy.Field()
    height = scrapy.Field()
    weight = scrapy.Field()
    url = scrapy.Field()

In [0]:
os.chdir('spiders')
!ls -l

total 16
-rw-r--r-- 1 root root  161 Jan 12 08:28 __init__.py
drwxr-sr-x 2 root root 4096 Jan 12 11:31 __pycache__
-rw-r--r-- 1 root root 4713 Jan 12 11:30 sprtprsn_scrape.py


In [0]:
%%writefile sprtprsn_scrape.py
import scrapy
from sprtprsn.items import sprtprsnItem
import re


class male_sprtprsn(scrapy.Spider):
    name = "male-sprtprsn"
    # first start url
    start_urls = ["https://healthyceleb.com/category/statistics/sports-stars/male-sports-stars"]

    npages = 7

    for i in range(2, npages + 2):
        start_urls.append(
            "https://healthyceleb.com/category/statistics/sports-stars/male-sports-stars/page/" + str(i) + "")

    def parse(self, response):
        for href in response.xpath("//div[contains(@class,'td-ss-main-content')]//a//@href"):
            url = href.extract()
            yield scrapy.Request(url, callback=self.parse_dir_contents)

    def parse_dir_contents(self, response):
        item = sprtprsnItem()
        data = response.xpath("//p/text()").extract()
        head = response.xpath("//h1/text()").extract()[0].strip()
        reg_n = re.compile(r'^([\w+\W+\s]+)\sHeight.*')
        reg_h = re.compile(r'.*\s([\d+.]+)\scm.*')
        reg_w = re.compile(r'^([\d+.]+)\skg.*')
        name = reg_n.match(head)
        item['iD'] = response.xpath("//meta[@property='og:url']/@content").extract()[0].split('/')[-1]
        item['name'] = name.group(1)
        #         item['name'] = response.xpath("//div[contains(@class,'td-post-content')]/p/strong/text()").extract()[0].strip()
        item['gender'] = 'Male'
        #         heighttext= response.xpath("//tr[contains(@class,'row-2 even')]/td[contains(@class,'column-2')]/text()").extract()[0].split()
        #         item['height'] = (int(heighttext[0]) * 30.48) + (int(heighttext[2]) * 2.54)
        #         weighttext = response.xpath("//tr[contains(@class,'row-3 odd')]/td[contains(@class,'column-2')]/text()").extract()[0].split()
        #         item['weight'] = weighttext[0]
        #         item['height'] = response.xpath("//tr[contains(@class,'row-2 even')]/td[contains(@class,'column-2')]/text()").extract()[0].strip()
        #         item['weight'] = response.xpath("//tr[contains(@class,'row-3 odd')]/td[contains(@class,'column-2')]/text()").extract()[0].strip()
        for p in data:
            if reg_h.match(p):
                h = reg_h.match(p)
                item['height'] = float(h.group(1))
            if reg_w.match(p):
                w = reg_w.match(p)
                item['weight'] = float(w.group(1))

        item['url'] = response.xpath("//meta[@property='og:url']/@content").extract()
        yield item


class female_sprtprsn(scrapy.Spider):
    name = "female-sprtprsn"
    # first start url
    start_urls = ["https://healthyceleb.com/category/statistics/sports-stars/female-sports-stars"]

    npages = 7

    for i in range(2, npages + 2):
        start_urls.append(
            "https://healthyceleb.com/category/statistics/sports-stars/female-sports-stars/page/" + str(i) + "")

    def parse(self, response):
        for href in response.xpath("//div[contains(@class,'td-ss-main-content')]//a//@href"):
            url = href.extract()
            yield scrapy.Request(url, callback=self.parse_dir_contents)

    def parse_dir_contents(self, response):
        item = sprtprsnItem()
        data = response.xpath("//p/text()").extract()
        head = response.xpath("//h1/text()").extract()[0].strip()
        reg_n = re.compile(r'^([\w+\W+\s]+)\sHeight.*')
        reg_h = re.compile(r'.*\s([\d+.]+)\scm.*')
        reg_w = re.compile(r'^([\d+.]+)\skg.*')
        name = reg_n.match(head)
        item['iD'] = response.xpath("//meta[@property='og:url']/@content").extract()[0].split('/')[-1]
        item['name'] = name.group(1)
        item['gender'] = 'Female'
        #         heighttext = response.xpath("//tr[contains(@class,'row-2 even')]/td[contains(@class,'column-2')]/text()").extract()[0].split()
        #         item['height'] = (int(heighttext[0]) * 30.48) + (int(heighttext[2]) * 2.54)
        #         weighttext = response.xpath("//tr[contains(@class,'row-3 odd')]/td[contains(@class,'column-2')]/text()").extract()[0].split()
        #         item['weight'] = weighttext[0]
        #         item['height'] = response.xpath("//tr[contains(@class,'row-2 even')]/td[contains(@class,'column-2')]/text()").extract()[0].strip()
        #         item['weight'] = response.xpath("//tr[contains(@class,'row-3 odd')]/td[contains(@class,'column-2')]/text()").extract()[0].strip()
        for p in data:
            if reg_h.match(p):
                h = reg_h.match(p)
                item['height'] = float(h.group(1))
            if reg_w.match(p):
                w = reg_w.match(p)
                item['weight'] =float(w.group(1))
        
        item['url'] = response.xpath("//meta[@property='og:url']/@content").extract()
        yield item

Overwriting sprtprsn_scrape.py


In [0]:
os.chdir('..')
!rm male_sprtprsn.csv
!rm female_sprtprsn.csv
!scrapy crawl male-sprtprsn -o male_sprtprsn.csv
!scrapy crawl female-sprtprsn -o female_sprtprsn.csv

2019-01-12 16:48:44 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: sprtprsn)
2019-01-12 16:48:44 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.0j  20 Nov 2018), cryptography 2.4.2, Platform Linux-4.14.79+-x86_64-with-Ubuntu-18.04-bionic
2019-01-12 16:48:44 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'sprtprsn', 'FEED_FORMAT': 'csv', 'FEED_URI': 'male_sprtprsn.csv', 'NEWSPIDER_MODULE': 'sprtprsn.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['sprtprsn.spiders']}
2019-01-12 16:48:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-01-12 16:48:44 [scrapy.middleware] INFO: Enabled downlo

In [0]:
import pandas as pd
import numpy as np
male = pd.read_csv("male_sprtprsn.csv")
# male.shape
male = male.dropna()
# male.shape
female = pd.read_csv("female_sprtprsn.csv")
# female.shape
female = female.dropna()
# female.shape

In [0]:
male100 = male.head(n = 100)
female100 = female.head(n = 100)

In [0]:
data = male100
data = data.append(female100,ignore_index=True)
data.shape

(200, 6)

In [0]:
data.head()


,gender,height,iD,name,url,weight
0,Male,188.0,118851,Ederson Moraes,https://healthyceleb.com/ederson-moraes-height...,83.0
1,Male,185.0,121956,Stephen James (Model),https://healthyceleb.com/stephen-james-model-h...,84.0
2,Male,180.0,121963,İlkay Gündoğan,https://healthyceleb.com/ilkay-gundogan-height...,78.0
3,Male,181.0,119341,Nicolás Otamendi,https://healthyceleb.com/nicolas-otamendi-heig...,74.0
4,Male,192.0,119692,Sergej Milinković-Savić,https://healthyceleb.com/sergej-milinkovic-sav...,76.0


In [0]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()
p = figure(title = "Height Vs Weight", x_axis_label='Height',y_axis_label='Weight',plot_width = 500, plot_height = 500)
p.circle(data['height'], data['weight'], size=5, color="black", alpha=0.5)
show(p,notebook_handle=True)

Loading BokehJS ...

In [0]:
data[data['weight'] == 180.0].url

Series([], Name: url, dtype: object)

In [0]:
data.sort_values('height',inplace=True)
data.head()

,gender,height,iD,name,url,weight
122,Female,137.0,90697,Isabella Cuda,https://healthyceleb.com/isabella-cuda-height-...,44.0
171,Female,142.0,62174,Simone Biles,https://healthyceleb.com/simone-biles-height-w...,46.0
173,Female,150.0,62233,Shawn Johnson,https://healthyceleb.com/shawn-johnson-height-...,45.0
120,Female,151.0,89883,Kristi Yamaguchi,https://healthyceleb.com/kristi-yamaguchi-heig...,45.0
194,Female,155.0,84687,Sofie Dossi,https://healthyceleb.com/sofie-dossi-height-we...,45.0


In [0]:
maleHeightStats = data[data['gender']=='Male'].height.describe()
list =[[maleHeightStats['min'],maleHeightStats['25%'],maleHeightStats['50%'],maleHeightStats['mean'],maleHeightStats['75%'],maleHeightStats['max']]]
maleHeightStatsDf = pd.DataFrame(list,columns = ['Minimum','1st Quartile','Median','Mean','3rd Quartile','Maximum'])
print("Male Heights\n",maleHeightStatsDf)

Male Heights
    Minimum  1st Quartile  Median    Mean  3rd Quartile  Maximum
0    160.0         176.0  181.75  182.16         188.0    208.0


In [0]:
femaleHeightStats = data[data['gender']=='Female'].height.describe()
list =[[femaleHeightStats['min'],femaleHeightStats['25%'],femaleHeightStats['50%'],femaleHeightStats['mean'],femaleHeightStats['75%'],femaleHeightStats['max']]]
femaleHeightStatsDf = pd.DataFrame(list,columns = ['Minimum','1st Quartile','Median','Mean','3rd Quartile','Maximum'])
print("Female Heights\n",femaleHeightStatsDf)

Female Heights
    Minimum  1st Quartile  Median    Mean  3rd Quartile  Maximum
0    137.0         165.0   170.0  170.22         177.0    196.0


In [0]:
maleWeightStats = data[data['gender']=='Male'].weight.describe()
list =[[maleWeightStats['min'],maleWeightStats['25%'],maleWeightStats['50%'],maleWeightStats['mean'],maleWeightStats['75%'],maleWeightStats['max']]]
maleWeightStatsDf = pd.DataFrame(list,columns = ['Minimum','1st Quartile','Median','Mean','3rd Quartile','Maximum'])
print("Male Weights\n",maleWeightStatsDf)

Male Weights
    Minimum  1st Quartile  Median  Mean  3rd Quartile  Maximum
0     57.0          70.0    75.0  77.5          83.0    127.0


In [0]:
femaleWeightStats = data[data['gender']=='Female'].weight.describe()
list =[[femaleWeightStats['min'],femaleWeightStats['25%'],femaleWeightStats['50%'],femaleWeightStats['mean'],femaleWeightStats['75%'],femaleWeightStats['max']]]
femaleWeightStatsDf = pd.DataFrame(list,columns = ['Minimum','1st Quartile','Median','Mean','3rd Quartile','Maximum'])
print("Female Weights\n",femaleWeightStatsDf)

Female Weights
    Minimum  1st Quartile  Median   Mean  3rd Quartile  Maximum
0     44.0          54.0    60.0  60.04          66.0     85.0


In [0]:
output_notebook()
p = figure(title = "Height Vs Weight", x_axis_label='Height',y_axis_label='Weight',plot_width = 500, plot_height = 500)
p.circle(data[data['gender']=='Male'].height, data[data['gender']=='Male'].weight, size=7, color="cyan", alpha=0.3,legend='Male')
p.circle(data[data['gender']=='Female'].height, data[data['gender']=='Female'].weight, size=7, color="red", alpha=0.3,legend='Female')
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p,notebook_handle=True)

Loading BokehJS ...

In [0]:
from sklearn.linear_model import LinearRegression
m_height, m_weight= np.array(data[data['gender']=='Male'].height),np.array(data[data['gender']=='Male'].weight)
m_height = m_height.reshape(-1,1)
m_lr = LinearRegression().fit(m_height, m_weight)

In [0]:
print("m_lr.coef_: %s" %m_lr.coef_)
print("m_lr.intercept_: %s" %m_lr.intercept_)

m_lr.coef_: [1.01149094]
m_lr.intercept_: -106.75319028223521


In [0]:
f_height, f_weight= np.array(data[data['gender']=='Female'].height),np.array(data[data['gender']=='Female'].weight)
f_height = f_height.reshape(-1,1)
f_lr = LinearRegression().fit(f_height, f_weight)

In [0]:
print("f_lr.coef_: %s" %f_lr.coef_)
print("f_lr.intercept_: %s" %f_lr.intercept_)

f_lr.coef_: [0.69009401]
f_lr.intercept_: -57.42780244033957


In [0]:
output_notebook()
p = figure(title = "Height Vs Weight", x_axis_label='Height',y_axis_label='Weight',plot_width = 500, plot_height = 500)
p.circle(data[data['gender']=='Male'].height, data[data['gender']=='Male'].weight, size=7, color="cyan", alpha=0.3,legend='Male')
p.circle(data[data['gender']=='Female'].height, data[data['gender']=='Female'].weight, size=7, color="red", alpha=0.3,legend='Female')
p.line(data[data['gender']=='Male'].height,m_lr.predict(m_height),color = 'black',legend='Male')
p.line(data[data['gender']=='Female'].height,f_lr.predict(f_height),color = 'black',legend='Female')
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p,notebook_handle=True)

Loading BokehJS ...